In [ ]:
import sys
sys.path.append("../..")
import numpy as np
import os
import pandas as pd
import matplotlib.pyplot as plt

from utils.io import load_yaml

server = "local"
server = load_yaml("../../config/servers.yml")[server]

increment = 5

database = server["database"]["location"]
prefix = "autolaparo_single_frames/AutoLaparo_Task2"
motion_pickle = f"23_03_03_pre_processed_frame_increment_{increment}_frames_between_clips_1_log.pkl"
label_pickle = "laparoscope_motion_label.csv"

# from readme
motion_dict = {
    0: "static",
    1: "up",
    2: "down",
    3: "left",
    4: "right",
    5: "zoom-in",
    6: "zoom-out",
}

motion_df = pd.read_pickle(os.path.join(database, prefix, motion_pickle))

length = len(motion_df[motion_df["vid"] == 0])
# get last half
motion_df = motion_df.groupby("vid").tail(int((length - 1) / 2)).reset_index(drop=True)
motion_df = motion_df[::increment].dropna().reset_index(drop=True)

label_df = pd.read_csv(os.path.join(database, prefix, label_pickle))
label_df["Clip"] = label_df["Clip"].apply(lambda x: x - 1)

print(label_df)
print(motion_df)

In [ ]:
import torch

from kornia.geometry import convert_points_to_homogeneous, transform_points

from utils.processing import four_point_homography_to_matrix, image_edges

id = 15
id_df = motion_df[motion_df["vid"] == id]
print(motion_dict[label_df[label_df["Clip"] == id].iloc[0].Label])

shape = (320, 240)
img = torch.ones((1, 1, shape[1], shape[0]))

x = []
y = []

for i in range(len(id_df)):
    duv = torch.tensor(id_df.iloc[i].duv).unsqueeze(0)
    
    # mean = duv.mean(axis=-2)
    # x.append(mean[0, 0])
    # y.append(mean[0, 1])


    h = four_point_homography_to_matrix(image_edges(img), duv)
    # print(h)

    # convert_points_to_homogeneous(torch.tensor([shape[0]/2, shape[1]/2]))
    p = torch.tensor([int((shape[1] - 1)/2), int((shape[0] - 1)/2), 1], dtype=torch.float)

    # print(p)
    # print(h @ p)
    dp = p - h @ p
    x.append(dp[0, 0])
    y.append(dp[0, 1])


import matplotlib.pyplot as plt
plt.scatter(x, y)
plt.show()


# duv = torch.tensor(motion_df.iloc[0].duv).unsqueeze(0)
# h = four_point_homography_to_matrix(image_edges(img), duv)

# # convert_points_to_homogeneous(torch.tensor([shape[0]/2, shape[1]/2]))
# p = torch.tensor([int((shape[1] - 1)/2), int((shape[0] - 1)/2), 1], dtype=torch.float)

# # print(p)
# # print(h @ p)
# print(p - h @ p)


In [ ]:
motion_dict[label_df[label_df["Clip"] == 10].Label.values[0]]